In [7]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
from collections import deque
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=FutureWarning)
simplefilter(action="ignore", category=UserWarning)
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

import loadBar
from csv_parser import CSVParser
from globals import RESOURCE_FOLDER, STEPSIZES, OUTPUT_WINDOW, INPUT_WINDOW, OUTPUT_FORECAST, DELETEABLE_COLUMNS, ONE_HOT_COLUMNS
from markovSquares import apply_markov
from feature_engineer import FeatureEngineer
from exploring_data_functions import *

from searoutePointFinder import fill_with_proximity

In [90]:
parser = CSVParser(RESOURCE_FOLDER)
total_df = parser.retrieve_training_data()

KeyboardInterrupt: 

In [ ]:
total_df["time"] = pd.to_datetime(total_df['time'])

In [65]:
def map_indexes_within_timewindow(df, time_col, timewindow):
    # Initialize the dictionary to store the mappings
    index_map = {}    

    within_window = deque()
    
    # Iterate over the DataFrame by index and row
    for current_idx in range(len((df))):
        current_row = df.iloc[current_idx]
        current_time = current_row[time_col]
        
        # Remove outdated timestamps from the left of the deque
        while within_window and (current_time - within_window[0][1] > timewindow):
            index_map[within_window[0][0]] = within_window[-1][0]
            within_window.popleft()
        
        # If this is the last row, map all the remaining indexes to this row
        if current_idx == df.index[-1]:
            for idx, _ in within_window:
                index_map[idx] = current_idx
        
        # Add the current index and time to the deque
        within_window.append((current_idx, current_time))
    
    return index_map

In [ ]:
def calculate_time_diffs_within_window(df):
    result_list= []
    vessel_dfs = df.groupby('vesselId')
    j=0
    for _, vessel_df in vessel_dfs:
        j+=1
        loadBar.load_bar(len(vessel_dfs), j)
        
        vessel_df = vessel_df.sort_values(by='time')
        groups = vessel_df.groupby(pd.Grouper(key="time", freq='5D'))
        for _, group in groups:
                for i in range(len(group)):
                    current_row = group.iloc[0]
                    future_row = group.iloc[i]
                    new_row = current_row.copy()
                    new_row['next_latitude'] = future_row['latitude']
                    new_row['next_longitude'] = future_row['longitude']
                    new_row['time_diff'] = (future_row['time'] - current_row['time']).total_seconds()
                    result_list.append(new_row)
    
    print("Concating")
    result_df = pd.DataFrame(result_list)
    result_df['diff_seconds'] = result_df['time_diff']
    result_df['diff_minutes'] = result_df['diff_seconds'] / 60
    result_df['diff_hours'] = result_df['diff_seconds'] / 3600
    result_df['diff_days'] = result_df['diff_seconds'] / 86400
    
    # Identify the last row for each vesselId for separation
    last_rows = df.groupby('vesselId').tail(1)

    last_rows = last_rows.reset_index(drop=True)
    result_df
    
    return result_df, last_rows

In [ ]:
def calculate_time_diffs_with_coords(df):
    # Ensure the DataFrame is sorted by vesselId and time
    df = df.sort_values(by=['vesselId', 'time']).reset_index(drop=True)
    last_rows = df.groupby('vesselId').tail(1)
    # Calculate the time difference in seconds between consecutive times per vesselId
    df['time_diff'] = df.groupby('vesselId')['time'].diff().shift(-1)
    
    # Create columns for each time unit by converting from seconds
    df['diff_seconds'] = df['time_diff'].dt.total_seconds()
    df['diff_minutes'] = df['diff_seconds'] / 60
    df['diff_hours'] = df['diff_seconds'] / 3600
    df['diff_days'] = df['diff_seconds'] / 86400
    
    # Get the latitude and longitude of the next row within each vesselId group
    df['next_latitude'] = df.groupby('vesselId')['latitude'].shift(-1)
    df['next_longitude'] = df.groupby('vesselId')['longitude'].shift(-1)
    
    # Identify the last row for each vesselId
    
    
    # Remove the last rows from the main DataFrame
    df = df.drop(last_rows.index).reset_index(drop=True)
    last_rows = last_rows.reset_index(drop=True)
    
    # Drop the temporary 'time_diff' column
    df = df.drop(columns=['time_diff'])
    
    return df, last_rows

In [ ]:
vessel_df1 = total_df[total_df['vesselId'] == "61e9f3aeb937134a3c4bfe3d"]

vessel_df1 = vessel_df1.sort_values(by='time')

diction=map_indexes_within_timewindow(vessel_df1, "time", timedelta(days=5))

print(vessel_df1)
print(diction)


                       time    cog  sog  rot  heading  navstat  latitude  \
43956   2024-01-04 19:04:25   64.0  0.0    6        9        1   9.45853   
44274   2024-01-04 19:49:21  301.0  0.0    4      356        1   9.45838   
44442   2024-01-04 20:13:20  317.4  0.0   -1       11        1   9.45845   
45226   2024-01-04 21:46:28  127.0  0.1   -1      346        1   9.45844   
45710   2024-01-04 22:48:42   99.0  0.0    5      346        1   9.45845   
...                     ...    ...  ...  ...      ...      ...       ...   
1513020 2024-05-07 10:54:10  349.9  0.0    0      344        5  31.14645   
1513116 2024-05-07 11:00:12  148.1  0.0    0      344        5  31.14645   
1519897 2024-05-07 23:15:16  323.3  0.0    0      344        5  31.14648   
1520076 2024-05-07 23:27:20  114.0  0.0    0      344        5  31.14648   
1520268 2024-05-07 23:48:16  179.6  0.0    0      344        5  31.14647   

         longitude                  vesselId                    portId  ...  \
43956   

In [108]:
total_df, last_rows = calculate_time_diffs_within_window(total_df)

KeyError: 'vesselId'

In [98]:
feature_engineering_functions = [categorize_navstat, numerize_UN_LOCODE, numerize_ISO] #, apply_markov, minutes_to_etaParsed, categorize_rot, type_dummies


feature_engineer = FeatureEngineer(total_df)
feature_engineer.apply_features(feature_engineering_functions)
total_df = feature_engineer.get_dataframe()

total_df.set_index("time", inplace=True)

In [99]:

def remove_non_numeric_columns(df):
    # Select columns that are either of type bool, float, or int
    df = df.select_dtypes(include=['bool', 'float', 'int'])
    return df

total_df = remove_non_numeric_columns(total_df)


In [106]:
print(len(total_df))

9338


In [101]:
def train_test_split(df_old, perc1, perc2):

    df = df_old.sample(frac=1).reset_index(drop=True)

    y_list = ['next_latitude', 'next_longitude'] #'next_latitude', 'next_longitude'
    ys = df[y_list]
    Xs = df.drop(columns = y_list)

    X_train = Xs.iloc[:int(np.round(Xs.shape[0]*perc1)),:]
    y_train = ys.iloc[:int(np.round(Xs.shape[0]*perc1)),:]
    X_val = Xs.iloc[int(np.round(Xs.shape[0]*perc1)):int(np.round(Xs.shape[0]*perc2)),:]
    y_val = ys.iloc[int(np.round(Xs.shape[0]*perc1)):int(np.round(Xs.shape[0]*perc2)),:]
    X_test = Xs.iloc[int(np.round(Xs.shape[0]*perc2)):,:]
    y_test = ys.iloc[int(np.round(Xs.shape[0]*perc2)):,:]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [102]:
X_train, y_train, X_val, y_val, X_test, y_test = train_test_split(total_df, 0.85, 0.99)

In [103]:
X_train_cols= X_train.columns

In [105]:
print(X_train)



dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest_X = xgb.DMatrix(X_test)

params = {"objective": "reg:squarederror",
            "max_depth": 7,
            "booster": "gbtree",
            "tree-method": "gpu_hist",
            "colsample_bynode": 0.4,
            "num_parallel_tree": 50,
            "subsample": 0.6,
            "seed": 42,
            "learning_rate": 1,
            #"n_estimators": 100,
            #"reg_alpha": 0.1,
            #"reg_lambda": 0.1,
            #"n_jobs": -1,
            "verbosity": 1
            }

#Subsample was 0.8, num-parallel tree was 100, colsample-bynode was 0.5 and max-depth was 5. Before tuning. 

#After first round of tuning we got sumbsample was 0.6, num-parallel tree was 50, colsample-bynode was 0.4 and max-depth was 7. Before tuning. 

num_boost_round = 30

#Tried 50, but that stagnated quickly. So reducing again to 30.  


early_stopping_rounds = 3

print(dtrain)

model = xgb.train(params, dtrain, num_boost_round, evals=[(dval, "validation")], 
                  early_stopping_rounds=early_stopping_rounds, verbose_eval=True)


preds = model.predict(dtest_X)

        cog   sog  rot  heading  navstat  latitude  longitude  portLongitude  \
0     286.0  16.5    0      286        0  36.30838   -7.52787      -6.922500   
1     254.0  13.0    0      253        0  50.12007   -2.13162      -1.967000   
2     133.0   0.0    0       40        5  35.89107   -5.50187      -5.399444   
3      25.0  15.8    0       29        0  43.37325   -9.65963      -9.167000   
4     204.0  17.5    0      203        0  25.42733   54.97281      55.059444   
...     ...   ...  ...      ...      ...       ...        ...            ...   
7932  179.0   0.0    0      165        5  51.32312    3.21045       3.207222   
7933  181.7  10.9    6      185        0 -37.88316  144.92815     144.917500   
7934  303.0   0.2    0      237        5  47.29233   -2.16240      -2.883000   
7935  149.8  12.9  -10      154        0 -30.32881  114.25141     115.850000   
7936   25.0  15.8    0       29        0  43.37325   -9.65963      -9.167000   

      portLatitude   CEU  ...  maxWidth

In [ ]:
def calculate_time_diff_with_last_rows(test_df, last_rows):
    # Merge test_df with last_rows on vesselId to get the last row time for each vesselId
    merged_df = pd.merge(
        test_df[["ID", "vesselId", "time"]],
        last_rows,
        on='vesselId',
        suffixes=('', '_last')
    )
    print(merged_df.columns)
    # Calculate time difference between test_df time and last row time
    merged_df['time_diff'] = (merged_df['time'] - merged_df['time_last']).dt.total_seconds()
    
    # Create columns for each time unit by converting from seconds
    merged_df['diff_seconds'] = merged_df['time_diff']
    merged_df['diff_minutes'] = merged_df['time_diff'] / 60
    merged_df['diff_hours'] = merged_df['time_diff'] / 3600
    merged_df['diff_days'] = merged_df['time_diff'] / 86400
    
    # Drop the temporary 'time_diff' column and 'time_last' if desired
    merged_df = merged_df.drop(columns=['time_diff', 'time_last'])
    
    
    return merged_df

In [ ]:
test_df = parser.retrieve_test_data()

In [ ]:
print(test_df.head())

   ID                  vesselId                time  scaling_factor
0   0  61e9f3aeb937134a3c4bfe3d 2024-05-08 00:03:16             0.3
1   1  61e9f473b937134a3c4c02df 2024-05-08 00:06:17             0.3
2   2  61e9f469b937134a3c4c029b 2024-05-08 00:10:02             0.3
3   3  61e9f45bb937134a3c4c0221 2024-05-08 00:10:34             0.3
4   4  61e9f38eb937134a3c4bfd8d 2024-05-08 00:12:27             0.3


In [ ]:
print(last_rows)

                   time    cog   sog  rot  heading  navstat  latitude  \
0   2024-05-05 13:25:50   40.0   0.0    1      328        1  36.84686   
1   2024-05-07 23:51:29  291.0   0.3    0      275        2  48.53320   
2   2024-05-07 21:53:14  129.3  15.6    0      128        0  28.82071   
3   2024-05-05 04:33:43  303.7  10.0   -2      303        0  33.86918   
4   2024-05-07 23:42:38   38.9  15.5    3       40        0  33.18068   
..                  ...    ...   ...  ...      ...      ...       ...   
683 2024-05-07 20:04:21   45.8   0.0    0      150        5 -34.06421   
684 2024-05-07 23:59:04   27.6  16.0    0       28        0  54.71940   
685 2024-05-01 11:34:29  182.0  15.0    0      182        0  33.96570   
686 2024-05-07 23:54:24   51.0   0.0    0       51        5  51.35306   
687 2024-05-07 23:59:01   53.6  17.7   -1       51        0  59.89167   

     longitude                   vesselId                    portId  ...  \
0      5.84638   61e9f38eb937134a3c4bfd8b  61d3

In [ ]:
test_df=calculate_time_diff_with_last_rows(test_df,last_rows)

Index(['ID', 'vesselId', 'time', 'time_last', 'cog', 'sog', 'rot', 'heading',
       'navstat', 'latitude', 'longitude', 'portId', 'etaParsed', 'UN_LOCODE',
       'ISO', 'portLongitude', 'portLatitude', 'shippingLineId', 'CEU', 'DWT',
       'GT', 'NT', 'vesselType', 'breadth', 'depth', 'draft', 'enginePower',
       'freshWater', 'fuel', 'homePort', 'length', 'maxHeight', 'maxSpeed',
       'maxWidth', 'rampCapacity', 'yearBuilt'],
      dtype='object')


In [ ]:
print(test_df.columns)

Index(['ID', 'vesselId', 'time', 'cog', 'sog', 'rot', 'heading', 'navstat',
       'latitude', 'longitude', 'portId', 'etaParsed', 'UN_LOCODE', 'ISO',
       'portLongitude', 'portLatitude', 'shippingLineId', 'CEU', 'DWT', 'GT',
       'NT', 'vesselType', 'breadth', 'depth', 'draft', 'enginePower',
       'freshWater', 'fuel', 'homePort', 'length', 'maxHeight', 'maxSpeed',
       'maxWidth', 'rampCapacity', 'yearBuilt', 'diff_seconds', 'diff_minutes',
       'diff_hours', 'diff_days'],
      dtype='object')


In [ ]:
feature_engineer = FeatureEngineer(test_df)
feature_engineer.apply_features(feature_engineering_functions)
test_df = feature_engineer.get_dataframe()

test_df.set_index("time", inplace=True)
test_df = remove_non_numeric_columns(test_df)



In [ ]:
print(test_df.head())

                     ID    cog   sog  rot  heading  navstat  latitude  \
time                                                                    
2024-05-08 00:03:16   0  179.6   0.0    0      344        5  31.14647   
2024-05-08 00:06:17   1   24.7   0.0    0      214        5  14.81694   
2024-05-08 00:10:02   2    8.0  18.7    0        6        0  38.27895   
2024-05-08 00:10:34   3  321.3   0.1    0       70        1 -43.53785   
2024-05-08 00:12:27   4  291.0   0.3    0      275        2  48.53320   

                     longitude  portLongitude  portLatitude  ...  maxSpeed  \
time                                                         ...             
2024-05-08 00:03:16  -81.49789     -81.496667     31.140556  ...       NaN   
2024-05-08 00:06:17  120.29625     120.279444     14.808333  ...       NaN   
2024-05-08 00:10:02   10.78280      11.780833     42.098889  ...       NaN   
2024-05-08 00:10:34  172.83522     172.716111    -43.606111  ...      22.2   
2024-05-08 00:12:27 

In [ ]:
def make_predictions(model, input_df):
    # Extract the features for prediction, excluding the 'ID' column
    features_df = input_df.drop(columns=['ID'])
    features_df = features_df[X_train_cols]
    # Generate predictions using the model
    predictions = model.predict(xgb.DMatrix(features_df))
    
    # Create a DataFrame with the predictions, renaming columns as required
    predictions_df = pd.DataFrame(predictions, columns=['next_latitude', 'next_longitude'])
    predictions_df = predictions_df.rename(columns={'next_latitude': 'latitude', 'next_longitude': 'longitude'})
    
    # Combine the 'ID' column with the predictions DataFrame
    result_df = pd.concat([input_df['ID'].reset_index(drop=True), predictions_df], axis=1)
    
    return result_df

In [ ]:
result_df = make_predictions(model, test_df)

In [ ]:
result_df.to_csv(RESOURCE_FOLDER+"/result_simple.csv", index=False)